## Classifier
load all the libraries


In [47]:
import sys
import numpy as np
import os
import os.path as op
import re
import pandas as pd
import imageio
from skimage.io import imread, imsave
from skimage.transform import resize
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import datetime
from PIL import Image
from itertools import chain
import csv
import random
import pickle

import warnings
warnings.filterwarnings('ignore');
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

plt.style.use('ggplot')
%matplotlib inline

define the function for evalation of the different Ml algorthim to check the accurauy and metrics of the dataset.<br>
load the data that was generate using the black box and the label file into one director having trainig and test 
data with labels

In [28]:
def evaluate_classifier(clf, X_test, y_test):
    y_pred_test = clf.predict(X_test)
    print("accuracy_score on test fit:",accuracy_score(y_test, y_pred_test))
    cm = confusion_matrix(y_test, y_pred_test)
    #cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("confusion matrix")
    print(cm)

In [22]:
#define the function loadDataAndLabels in to file
def loadDataAndLabels(myDataDir):
    imgFiles = []
    myLabel = []
    with open(myDataDir+'/aa-labels.csv','r') as fi:
        reader =  csv.reader(fi)
        ignoreHead = 0
        for row in reader:
            #print(row)
            if len(row) > 0:
                if ignoreHead == 0:
                    ignoreHead = 1
                else:
                    imgFiles.append(row[0])
                    myLabel.append(row[1])
    fi.close()
    num_ims = len(imgFiles)
    num_rows = 100
    num_cols = 100
    num_chans = 3
    resize_factor = 4
    #resize
    num_rows_resize = int(num_rows/resize_factor)
    num_cols_resize = int(num_cols/resize_factor)
    myDataNp = np.zeros((num_ims,num_rows_resize,num_cols_resize,num_chans))
    myLabelNp = np.array(myLabel)
    i = 0
    for imgName in imgFiles:
        img_data = resize(imread(myDataDir+"/"+imgName+".jpg"),(num_rows_resize,num_cols_resize),mode='constant')
        myDataNp[i,:,:,:] = img_data[:,:,:]
        if i % 1000 == 0:
            print(i,": loaded img - ", myDataDir,"/",imgName,".jpg")
        i = i+1
    #reshape
    myDataNp = myDataNp.reshape(num_ims,num_rows_resize*num_cols_resize*num_chans)
    return(myDataNp,myLabelNp)

In [23]:
TR_DATA_DIR = 'C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1'
# Load the training set
(trData,trLabel) = loadDataAndLabels(TR_DATA_DIR)

0 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Braeburn_0_100_f0 .jpg
1000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Golden 2_107_100_f7 .jpg
2000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Granny Smith_113_100_f1 .jpg
3000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Red 2_117_100_f3 .jpg
4000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Red Delicious_125_100_f2 .jpg
5000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Red Yellow 2_156_100_f7 .jpg
6000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Avocado_136_100_f2 .jpg
7000 : loaded img -  C:/Users/sonali/Deskto

In [25]:
TE_DATA_DIR = 'C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1'
# Load the test set
(teData,teLabel) = loadDataAndLabels(TE_DATA_DIR)

0 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Apple Braeburn_321_100_f0 .jpg
1000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Apple Red Yellow 2_112_100_f4 .jpg
2000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Carambula_146_100_f8 .jpg
3000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Granadilla_105_100_f0 .jpg
4000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Huckleberry_100_100_f11 .jpg
5000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Mangostan_132_100_f6 .jpg
6000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Pear_322_100_f1 .jpg
7000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_med

### Algorithim
Tried different algorithim to check the which will give the better performace. claculate the accuracy score and confusion matrix to check how many have misclassified.
Also save the model

In [32]:
#LogisticRegression
logreg = LogisticRegression()
#fit the model
logreg.fit(trData, trLabel)
#print confusion matrix for training and test dataset
print("Training Dataset")
print(evaluate_classifier(logreg, trData, trLabel))
print("Testing Dataset")
print(evaluate_classifier(logreg, teData, teLabel))

Training Dataset
accuracy_score on test fit: 0.6059252344757079
confusion matrix
[[15942   134   905   785   200]
 [ 2505   280  1173   608   112]
 [ 2625   131  3762  2231   264]
 [ 1696    58  1413  5696   534]
 [  522     5   159  1671  1583]]
None
Testing Dataset
accuracy_score on test fit: 0.5510880371147686
confusion matrix
[[3080   33  203  185   70]
 [ 475   36  205  146   32]
 [ 582   50  565  512  107]
 [ 382   18  337 1007  180]
 [ 123    2   44  378  301]]
None


In [57]:
# Save model..
filename = 'clf_logreg_medtraj1.sav'
pickle.dump(logreg, open(filename, 'wb'))
logreg_loaded = pickle.load(open(filename, 'rb'))
evaluate_classifier(logreg_loaded, teData, teLabel)

accuracy_score on test fit: 0.5510880371147686
confusion matrix
[[3080   33  203  185   70]
 [ 475   36  205  146   32]
 [ 582   50  565  512  107]
 [ 382   18  337 1007  180]
 [ 123    2   44  378  301]]


#### Gaussian_NB
accuracy score is very less

In [35]:
#Gaussian_NB
Gaussian_NB=GaussianNB()
#fit model
Gaussian_NB.fit(trData, trLabel)
#print confusion matrix for training and test dataset
print("Training Dataset")
print(evaluate_classifier(Gaussian_NB, trData, trLabel))
print("Testing Dataset")
print(evaluate_classifier(Gaussian_NB, teData, teLabel))


Training Dataset
accuracy_score on test fit: 0.28937191625550074
confusion matrix
[[7784 1745  252  294 7891]
 [ 593  926  171  136 2852]
 [ 616 1193  453  277 6474]
 [ 396  514  226  198 8063]
 [ 109   80   60   32 3659]]
None
Testing Dataset
accuracy_score on test fit: 0.29338340881475755
confusion matrix
[[1592  316   60   56 1547]
 [ 125  148   24   32  565]
 [ 143  219   84   43 1327]
 [  89   95   52   43 1645]
 [  23   17    9   10  789]]
None


In [56]:
# Save model..
filename = 'clf_Gaussian_NB_medtraj1.sav'
pickle.dump(Gaussian_NB, open(filename, 'wb'))
Gaussian_NB_loaded = pickle.load(open(filename, 'rb'))
evaluate_classifier(Gaussian_NB_loaded, teData, teLabel)

accuracy_score on test fit: 0.29338340881475755
confusion matrix
[[1592  316   60   56 1547]
 [ 125  148   24   32  565]
 [ 143  219   84   43 1327]
 [  89   95   52   43 1645]
 [  23   17    9   10  789]]


#### random forest

In [36]:
#random_forest
random_forest = RandomForestClassifier(n_estimators=10)
random_forest.fit(trData, trLabel)
#confusion matrix
print("Training Dataset")
print(evaluate_classifier(random_forest, trData, trLabel))
print("Testing Dataset")
print(evaluate_classifier(random_forest, teData, teLabel))

Training Dataset
accuracy_score on test fit: 0.9941547761923812
confusion matrix
[[17954     8     2     2     0]
 [   66  4592    17     3     0]
 [    9    15  8966    23     0]
 [    0     2    38  9345    12]
 [    0     0     1    65  3874]]
None
Testing Dataset
accuracy_score on test fit: 0.7563238705401525
confusion matrix
[[3399   92   65   14    1]
 [ 319  275  260   40    0]
 [ 130  181 1130  359   16]
 [  15   12  322 1434  141]
 [   2    0    4  233  609]]
None


In [49]:
# Save model..
filename = 'clf_randomforest_medtraj1.sav'
pickle.dump(random_forest, open(filename, 'wb'))

In [50]:
random_forest_loaded = pickle.load(open(filename, 'rb'))
evaluate_classifier(random_forest_loaded, teData, teLabel)

accuracy_score on test fit: 0.7563238705401525
confusion matrix
[[3399   92   65   14    1]
 [ 319  275  260   40    0]
 [ 130  181 1130  359   16]
 [  15   12  322 1434  141]
 [   2    0    4  233  609]]


### Support vextor classifier

In [40]:
#SVC
svc=SVC(C=1.0, kernel='rbf',degree=3,random_state=42)
svc.fit(trData, trLabel)
print("Training Dataset")
print(evaluate_classifier(svc, trData, trLabel))
print("Testing Dataset")
print(evaluate_classifier(svc, teData, teLabel))

Training Dataset
accuracy_score on test fit: 0.6990043116860026
confusion matrix
[[16677    30   860   394     5]
 [ 2045   234  1912   475    12]
 [ 1059    45  5357  2525    27]
 [  393     0  1239  7497   268]
 [   84     0     1  2169  1686]]
None
Testing Dataset
accuracy_score on test fit: 0.6854081519938142
confusion matrix
[[3330    5  157   78    1]
 [ 403   30  350  109    2]
 [ 255    9  971  574    7]
 [  73    0  242 1536   73]
 [  17    0    0  493  338]]
None


In [51]:
# Save model..
filename = 'clf_SVC_medtraj1.sav'
pickle.dump(svc, open(filename, 'wb'))
svc_loaded = pickle.load(open(filename, 'rb'))
evaluate_classifier(svc_loaded, teData, teLabel)

accuracy_score on test fit: 0.6854081519938142
confusion matrix
[[3330    5  157   78    1]
 [ 403   30  350  109    2]
 [ 255    9  971  574    7]
 [  73    0  242 1536   73]
 [  17    0    0  493  338]]


In [41]:
#Bagging classifier
bagging_classifier=BaggingClassifier(n_estimators=10,max_samples=1.0,random_state=42,)
bagging_classifier.fit(trData, trLabel)
print("Training Dataset")
print(evaluate_classifier(random_forest, trData, trLabel))
print("Testing Dataset")
print(evaluate_classifier(bagging_classifier, teData, teLabel))

Training Dataset
accuracy_score on test fit: 0.9941547761923812
confusion matrix
[[17954     8     2     2     0]
 [   66  4592    17     3     0]
 [    9    15  8966    23     0]
 [    0     2    38  9345    12]
 [    0     0     1    65  3874]]
None
Testing Dataset
accuracy_score on test fit: 0.749364851430465
confusion matrix
[[3352  123   73   19    4]
 [ 315  304  230   43    2]
 [ 119  190 1164  324   19]
 [  24   14  360 1376  150]
 [   4    1    5  250  588]]
None


In [55]:
# Save model..
filename = 'clf_bagging_classifier_medtraj1.sav'
pickle.dump(bagging_classifier, open(filename, 'wb'))
bagging_classifier_loaded = pickle.load(open(filename, 'rb'))
evaluate_classifier(bagging_classifier_loaded, teData, teLabel)

accuracy_score on test fit: 0.749364851430465
confusion matrix
[[3352  123   73   19    4]
 [ 315  304  230   43    2]
 [ 119  190 1164  324   19]
 [  24   14  360 1376  150]
 [   4    1    5  250  588]]


In [44]:
#AdaBoost_Classifier
adaBoost_classifier=AdaBoostClassifier(n_estimators=50,random_state=42)
adaBoost_classifier.fit(trData, trLabel)
#print confusion matrix
print("Training Dataset")
print(evaluate_classifier(adaBoost_classifier, trData, trLabel))
print("Testing Dataset")
print(evaluate_classifier(adaBoost_classifier, teData, teLabel))

Training Dataset
accuracy_score on test fit: 0.5922567453438237
confusion matrix
[[16056     2  1020   835    53]
 [ 3069     5  1091   482    31]
 [ 3721    10  3284  1843   155]
 [ 1708     0  1790  4990   909]
 [  285     0     2  1340  2313]]
None
Testing Dataset
accuracy_score on test fit: 0.5908538605986966
confusion matrix
[[3164    0  213  183   11]
 [ 543    0  217  122   12]
 [ 737    2  604  450   23]
 [ 332    0  330 1059  203]
 [  58    0    5  263  522]]
None


In [54]:
# Save model..
filename = 'clf_adaBoost_classifier_medtraj1.sav'
pickle.dump(adaBoost_classifier, open(filename, 'wb'))
adaBoost_classifier_loaded = pickle.load(open(filename, 'rb'))
evaluate_classifier(adaBoost_classifier_loaded, teData, teLabel)

accuracy_score on test fit: 0.5908538605986966
confusion matrix
[[3164    0  213  183   11]
 [ 543    0  217  122   12]
 [ 737    2  604  450   23]
 [ 332    0  330 1059  203]
 [  58    0    5  263  522]]


In [46]:
#Gradientboosting_classifier
gradientboosting_classifier=GradientBoostingClassifier(learning_rate=0.1,n_estimators=100,random_state=42)
gradientboosting_classifier.fit(trData, trLabel)
print("Training Dataset")
print(evaluate_classifier(gradientboosting_classifier, trData, trLabel))
print("Testing Dataset")
print(evaluate_classifier(gradientboosting_classifier, teData, teLabel))

Training Dataset
accuracy_score on test fit: 0.7590789883095523
confusion matrix
[[17178   118   420   232    18]
 [ 1992   849  1502   319    16]
 [  967   113  5988  1892    53]
 [  397     3  1065  7444   488]
 [   74     0     0  1171  2695]]
None
Testing Dataset
accuracy_score on test fit: 0.7184358776096321
confusion matrix
[[3397   33   79   54    8]
 [ 403   98  309   80    4]
 [ 253   46 1026  469   22]
 [  91    2  264 1420  147]
 [  15    0    1  269  563]]
None


In [53]:
# Save model..
filename = 'clf_gradientboosting_classifier_medtraj1.sav'
pickle.dump(gradientboosting_classifier, open(filename, 'wb'))
gradientboosting_classifier_loaded = pickle.load(open(filename, 'rb'))
evaluate_classifier(gradientboosting_classifier_loaded, teData, teLabel)

accuracy_score on test fit: 0.7184358776096321
confusion matrix
[[3397   33   79   54    8]
 [ 403   98  309   80    4]
 [ 253   46 1026  469   22]
 [  91    2  264 1420  147]
 [  15    0    1  269  563]]


In [48]:
#DecisionTree_Classifier
DecisionTree_Classifier=DecisionTreeClassifier(criterion='gini',splitter='best',max_depth=5,random_state=42)
DecisionTree_Classifier.fit(trData, trLabel)
print("Training Dataset")
print(evaluate_classifier(DecisionTree_Classifier, trData, trLabel))
print("Testing Dataset")
print(evaluate_classifier(DecisionTree_Classifier, teData, teLabel))

Training Dataset
accuracy_score on test fit: 0.5917455660754767
confusion matrix
[[15298     0  1741   913    14]
 [ 2071     0  2017   567    23]
 [ 1915     0  4302  2721    75]
 [  906     0  2136  5866   489]
 [  162     0   232  2387  1159]]
None
Testing Dataset
accuracy_score on test fit: 0.5713023307190986
confusion matrix
[[3014    0  364  192    1]
 [ 390    0  361  138    5]
 [ 433    0  778  587   18]
 [ 202    0  440 1151  131]
 [  37    0   46  536  229]]
None


In [52]:
# Save model..
filename = 'clf_DecisionTree_Classifier_medtraj1.sav'
pickle.dump(DecisionTree_Classifier, open(filename, 'wb'))
DecisionTree_Classifier_loaded = pickle.load(open(filename, 'rb'))
evaluate_classifier(DecisionTree_Classifier_loaded, teData, teLabel)

accuracy_score on test fit: 0.5713023307190986
confusion matrix
[[3014    0  364  192    1]
 [ 390    0  361  138    5]
 [ 433    0  778  587   18]
 [ 202    0  440 1151  131]
 [  37    0   46  536  229]]
